[View in Colaboratory](https://colab.research.google.com/github/geotana/UdacityDL/blob/master/6_lstm_TB_4Win1.ipynb)

Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [9]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [10]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0]) #unicode

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), "|", char2id('z'), "|", char2id(' '), "|", char2id('ï'))
print(id2char(1),"|",  id2char(26), "|", id2char(0))

Unexpected character: ï
1 | 26 | 0 | 0
a | z |  


Function to generate a training batch for the LSTM model.

In [11]:
batch_size= 64 #64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)] #横にprobabilitiesの0軸要素分ならぶ

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0] # bs分並んだ空のリスト
  for b in batches: #bはunr
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'of the orig']
['ists advoca', 'ginal docum']
[' a']
['an']


In [0]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [0]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
#   with tf.name_scope("Input"):
#     ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1),name="ix")
#     im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1),name="im")
#     ib = tf.Variable(tf.zeros([1, num_nodes]),name="ib")
    
#   # Forget gate: input, previous output, and bias.
#   with tf.name_scope("Forget"):
#     fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
#     fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
#     fb = tf.Variable(tf.zeros([1, num_nodes]))
    
#   # Memory cell: input, previous state and bias.                             
#   with tf.name_scope("Memory"):
#     cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
#     cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
#     cb = tf.Variable(tf.zeros([1, num_nodes]))
    
#   # Output gate: input, previous output, and bias.
#   with tf.name_scope("Output"):
#     ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
#     om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
#     ob = tf.Variable(tf.zeros([1, num_nodes]))

  big_x = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes * 4], -0.1, 0.1))
  big_m = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1))
  big_b = tf.Variable(tf.zeros([1, num_nodes * 4]))
    
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False,name="saved_output")
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False,name="saved_state")
  
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1),name="w")
  b = tf.Variable(tf.zeros([vocabulary_size]),name="b")
  
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state): #receiving input, hidden(t-1) and state(t-1)
    
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    
    with tf.name_scope("lstm_cell"):
  
      split0, split1, split2, split3 = tf.split(tf.matmul(i, big_x) + tf.matmul(o, big_m) + big_b, num_or_size_splits=4, axis=1)
      input_gate = tf.sigmoid(split0)
      forget_gate = tf.sigmoid(split1)
      update = split2

#       input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
#       forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
#       update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
      
      state = forget_gate * state + input_gate * tf.tanh(update) # state(t-1) --> state(t)
      
#       output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
      output_gate = tf.sigmoid(split3)
  
    return output_gate * tf.tanh(state), state #returning output(t) and state(t)
 



  # Input data.
  train_data = list()
  with tf.name_scope("train_data"):
    for _ in range(num_unrollings + 1):
      train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.
#   num_unrollings = 3  ->  a, b, c, d 
  # train_inputs = a, b, c  　0番目から2番目
  # train_labels = b, c, d　 　1番目から3番目(最後)まで
  
  

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  with tf.name_scope("train_inputs"):
    for i in train_inputs: #ここで　train_inputsの数分だけセルを作る
      output, state = lstm_cell(i, output, state)
      outputs.append(output)

  # State saving across unrollings. logitsとlossの前にsaveをする
  with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b) #次元あっているのYes
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=tf.concat(train_labels, 0), logits=logits),name="loss") #run

  # Optimizer.
  global_step = tf.Variable(0)
  with tf.name_scope("Optimize"):
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step) #run

  # Predictions.
  train_prediction = tf.nn.softmax(logits,name="Predictions") #runされる
  
  # Sampling and validation eval: batch 1, no unrolling.　なぜunroで精度を上げない？
  with tf.name_scope("Optimize"):    
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
      saved_sample_output.assign(tf.zeros([1, num_nodes])),
      saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(
      sample_input, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
      sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
  
  #TensorBoard
  tf.summary.scalar('Loss', loss) # yをスカラーとして記録
  merged = tf.summary.merge_all() # おまじない
  if tf.gfile.Exists('./logdir'):
    tf.gfile.DeleteRecursively('./logdir') # ./logdirが存在する場合削除
  writer = tf.summary.FileWriter('./logdir',graph) # 保存先を./logdirに設定


In [18]:
num_steps = 7001 #7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    summary, _, l, predictions, lr = session.run(
      [merged, optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    
    #predictions = np 80,128   80 = bs x unr 　を np(bs, em)に分解 
    pred_batches = list()  #   bs x em のリスト
    for unr in range(num_unrollings) : 
      pred_batches.append(predictions[0 + unr : batch_size + unr,:])
     

    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
    
      writer.add_summary(summary, step) #TB
      
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution()) #ランダムな1-hot-vecの1文字
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
              #トレーニングデータを印刷
        print("inp",batches2string( batches[0:num_unrollings]), "ans",batches2string( batches[1:]), 
          "pre", batches2string(pred_batches) ) 

        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
inp ['perty is t', 'e t four r'] ans ['erty is th', ' t four re'] pre ['mhglqhuhup', 'hglqhuhuph']
Average loss at step 0: 3.295092 learning rate: 10.000000
Minibatch perplexity: 26.98
jt  ftg r  rwdhewyfvhns zge rr   rohqngtd qk fp a rdokqty ibii b r lps nytt rvi 
huo z uemef tt ri heto dh trx ehth dl rz gt  t  ct krfmvnwmr ercrj   ri cmtscqut
bdkztrknz  td duanx  k lrth j iyl henutse btkxfjeneftt hc rrhfimdfsx  t  msi sqt
b mqnl  e  jye aejit tbls jhsgagysowrhgth ws tmm rj t wbvuvnmv q  q  ld xdpet  q
vspiukp  lfttett  kfb yum y wozftitglmt t me  t h   iwe t hqurt xtstn  w   b   k
Validation set perplexity: 22.80
inp ['heft in th', 'ecommendat'] ans ['eft in thi', 'commendati'] pre ['          ', '          ']
inp ['is work he', 'ion class '] ans ['s work he ', 'on class c'] pre ['tttttttttt', 'tttttttttt']
inp [' opposed t', 'computer m'] ans ['opposed th', 'omputer mo'] pre ['          ', '          ']
inp ['he institu', 'odems are '] ans ['e institut', 'dems are o'] pr

inp ['ed egoism ', 'e space th'] ans ['d egoism a', ' space thi'] pre ['   aa re t', '  aa re t ']
inp ['and a form', 'is minimis'] ans ['nd a form ', 's minimise'] pre ['std  aaest', 'td  aaests']
inp [' of amoral', 'es the tra'] ans ['of amorali', 's the tran'] pre ['m f  mmhni', ' f  mmhnii']
inp ['ism in whi', 'nsmission '] ans ['sm in whic', 'smission t'] pre ['ns  aaann ', 's  aaann d']
inp ['ch individ', 'time of mo'] ans ['h individu', 'ime of mos'] pre ['ihissin ss', 'hissin ss ']
inp ['uals would', 'st faxes e'] ans ['als would ', 't faxes ea'] pre ['m niim omn', ' niim omnh']
inp [' unite in ', 'ach scanne'] ans ['unite in a', 'ch scanned'] pre ['asfe esa e', 'sfe esa e ']
inp ['associatio', 'd line is '] ans ['ssociation', ' line is c'] pre ['n  a ennen', '  a ennenn']
inp ['ns of egoi', 'compressed'] ans ['s of egois', 'ompressed '] pre [' i siisi i', 'i siisi ii']
inp ['sts only w', ' independe'] ans ['ts only wh', 'independen'] pre ['soessso fs', 'oessso fss']
inp ['hen 

 ['archist pe', 'or higher '] ans ['rchist per', 'r higher b'] pre ['tn  oa iss', 'n  oa iss ']
inp ['riodical e', 'but fax ma'] ans ['iodical ev', 'ut fax mac'] pre [' isrnh bsi', 'isrnh bsih']
inp ['ver issued', 'chines sup'] ans ['er issued ', 'hines supp'] pre ['ia i cads ', 'a i cads  ']
inp [' in the en', 'port speed'] ans ['in the eng', 'ort speeds'] pre ['pesus i ep', 'esus i epe']
inp ['glish lang', 's beginnin'] ans ['lish langu', ' beginning'] pre ['h epnesne ', ' epnesne p']
inp ['uage tucke', 'g with two'] ans ['age tucker', ' with two '] pre ['nhnn innn ', 'hnn innn h']
inp ['r s concep', ' four zero'] ans [' s concept', 'four zero '] pre [' tti utgo ', 'tti utgo  ']
inp ['tion of in', ' zero bit '] ans ['ion of ind', 'zero bit s'] pre [' tcenrgot ', 'tcenrgot n']
inp ['dividualis', 's the tran'] ans ['ividualist', ' the trans'] pre ['ioooiotei ', 'oooiotei o']
inp ['t anarchis', 'sferred im'] ans [' anarchism', 'ferred ima'] pre ['  tinrditi', ' tinrditii']
inp ['m incor

inp ['e between ', ' a fax is '] ans [' between t', 'a fax is s'] pre ['nffnefrien', 'ffnefriene']
inp ['the camps ', 'sending an'] ans ['he camps w', 'ending an '] pre ['hee   f an', 'ee   f ant']
inp ['was over s', ' email wit'] ans ['as over st', 'email with'] pre ['aann  annn', 'ann  annn ']
inp ['trategy th', 'h one or m'] ans ['rategy the', ' one or mo'] pre ['ieessf er ', 'eessf er  ']
inp ['e anarchis', 'ore image '] ans [' anarchist', 're image f'] pre [' mt    tts', 'mt    tts ']
inp ['ts around ', 'files as a'] ans ['s around b', 'iles as at'] pre ['ea raerre ', 'a raerre r']
inp ['bakunin fa', 'ttachments'] ans ['akunin fav', 'tachments '] pre ['a s  sra e', ' s  sra es']
inp ['voured in ', ' this allo'] ans ['oured in k', 'this allow'] pre ['earhne nnt', 'arhne nnt ']
inp ['kropotkin ', 'ws color a'] ans ['ropotkin s', 's color an'] pre ['i o ua orl', ' o ua orl ']
inp ['s words di', 'nd is more'] ans [' words dir', 'd is more '] pre ['  o oorn  ', ' o oorn  o']
inp ['rect

inp ['read and f', ' pioneer i'] ans ['ead and fi', 'pioneer in'] pre ['ooer o  t ', 'oer o  t  ']
inp ['ields fact', 'n mechanic'] ans ['elds facto', ' mechanica'] pre ['nenfeaeeeo', 'enfeaeeeof']
inp ['ories and ', 'al image s'] ans ['ries and w', 'l image sc'] pre ['rnalnadn  ', 'nalnadn  a']
inp ['workshops ', 'canning an'] ans ['orkshops h', 'anning and'] pre ['aarn  a  a', 'arn  a  ae']
inp ['he felt co', 'd transmis'] ans ['e felt co ', ' transmiss'] pre ['a nhahaann', ' nhahaannn']
inp [' operation', 'sion in on'] ans ['operation ', 'ion in one'] pre ['s fsrn   s', ' fsrn   ss']
inp [' is more b', 'e nine eig'] ans ['is more be', ' nine eigh'] pre ['o no eonoe', ' no eonoen']
inp ['eneficial ', 'ht five dr'] ans ['neficial t', 't five dr '] pre [' ei  iiinr', 'ei  iiinr ']
inp ['than compe', ' hank magn'] ans ['han compet', 'hank magnu'] pre ['hfeett ii ', 'feett ii o']
inp ['tition ill', 'uski found'] ans ['ition illu', 'ski founde'] pre [' no h onnh', 'no h onnh ']
inp ['ustr

...

inp ['sm makes n', 'the old or'] ans ['m makes no', 'he old ort'] pre [' haea aanr', 'haea aanre']
inp ['o mention ', 'thography '] ans [' mention o', 'hography o'] pre ['t aeasaite', ' aeasaite ']
inp ['of level o', 'of irish t'] ans ['f level of', 'f irish th'] pre ['nf  ttonae', 'f  ttonaee']
inp ['f intellec', 'he french '] ans [' intellect', 'e french f'] pre [' et na oeo', 'et na oeor']
inp ['tual funct', 'franc can '] ans ['ual functi', 'ranc can b'] pre ['ioroln toe', 'oroln toeo']
inp ['ioning but', 'be written'] ans ['oning but ', 'e written '] pre ['oen  onhgo', 'en  onhgo ']
inp [' the fact ', ' ff or uni'] ans ['the fact t', 'ff or unic'] pre ['othoeo iaf', 'thoeo iafo']
inp ['that asper', 'code u two'] ans ['hat asperg', 'ode u two '] pre ['haente rtt', 'aente rttn']
inp ['ger s auti', ' zero a th'] ans ['er s autis', 'zero a thr'] pre ['ef e rto  ', 'f e rto  a']
inp ['stics as a', 'ree in mat'] ans ['tics as a ', 'ee in math'] pre ['teiec  f n', 'eiec  f na']
Average lo

inp [' triggers ', 'cut some p'] ans ['triggers t', 'ut some pr'] pre ['aohraigaho', 'ohraigahor']
inp ['the disord', 'reservatio'] ans ['he disorde', 'eservation'] pre ['hoes ea i ', 'oes ea i s']
inp ['er this tr', 'n methods '] ans ['r this tri', ' methods r'] pre ['r  aaahaeo', '  aaahaeos']
inp ['iggering c', 'require th'] ans ['ggering co', 'equire the'] pre ['gehvru r l', 'ehvru r ln']
inp ['ould occur', 'e food to '] ans ['uld occur ', ' food to b'] pre ['n rado rtr', ' rado rtrf']
inp [' during ge', 'be sealed '] ans ['during ges', 'e sealed a'] pre ['tei stienv', 'ei stienvg']
inp ['station du', 'after trea'] ans ['tation due', 'fter treat'] pre [' n  neiro ', 'n  neiro n']
inp ['e to a tox', 'tment to p'] ans [' to a toxi', 'ment to pr'] pre [' itehn tt ', 'itehn tt n']
inp ['in that en', 'revent re '] ans ['n that ent', 'event re c'] pre ['negstehre ', 'egstehre t']
inp ['ters the m', 'contaminat'] ans ['ers the mo', 'ontaminati'] pre ['rorn t rtt', 'orn t rtth']
inp ['othe

inp ['e below th', ' preservin'] ans [' below the', 'preserving'] pre [' taree sae', 'taree saer']
inp ['e answer t', 'g fish and'] ans [' answer to', ' fish and '] pre ['  aanodv  ', ' aanodv  a']
inp ['o this que', ' especiall'] ans [' this ques', 'especially'] pre [' atxheersr', 'atxheersr ']
inp ['stion has ', 'y herrings'] ans ['tion has s', ' herrings '] pre ['   anina e', '  anina ea']
inp ['significan', ' in this w'] ans ['ignificant', 'in this wa'] pre ['eaxnh  tnh', 'axnh  tnhe']
inp ['t ramifica', 'ay freezin'] ans [' ramificat', 'y freezing'] pre [' ra etnoio', 'ra etnoion']
inp ['tions on t', 'g is also '] ans ['ions on th', ' is also o'] pre ['i oann   a', ' oann   aa']
inp ['he directi', 'one of the'] ans ['e directio', 'ne of the '] pre ['ef ei ifsf', 'f ei ifsfe']
inp ['on of rese', ' most comm'] ans ['n of resea', 'most commo'] pre ['ni aarft  ', 'i aarft  t']
inp ['arch since', 'only used '] ans ['rch since ', 'nly used p'] pre ['rn  h   ia', 'n  h   iae']
inp [' a r

inp ['ince the c', 't in brand'] ans ['nce the ca', ' in brandy'] pre ['x goens ot', ' goens oth']
inp ['ause of au', 'y or other'] ans ['use of aut', ' or other '] pre ['n siefd it', ' siefd itf']
inp ['tism is th', ' spirits t'] ans ['ism is tho', 'spirits th'] pre ['iosumr loe', 'osumr loen']
inp ['ought to b', 'hese shoul'] ans ['ught to be', 'ese should'] pre ['seg he   i', 'eg he   io']
inp ['e at least', 'd not be c'] ans [' at least ', ' not be co'] pre ['  otso to ', ' otso to a']
inp [' partly ge', 'onfused wi'] ans ['partly gen', 'nfused wit'] pre ['omrtratr e', 'mrtratr ey']
inp ['netic a pr', 'th fruit f'] ans ['etic a pro', 'h fruit fl'] pre ['thr  too o', 'hr  too ot']
inp ['oportion o', 'lavoured s'] ans ['portion of', 'avoured sp'] pre ['coucri r s', 'oucri r sc']
inp ['f these ad', 'pirits suc'] ans [' these adu', 'irits such'] pre [' etlheen i', 'etlheen ie']
inp ['ults seek ', 'h as cherr'] ans ['lts seek t', ' as cherry'] pre ['r tt s  to', ' tt s  tou']
inp ['thei

inp ['ometimes s', 'ottle has '] ans ['metimes su', 'ttle has b'] pre ['moe  e en ', 'oe  e en e']
inp ['uccessful ', 'been opene'] ans ['ccessful a', 'een opened'] pre ['ceh iret a', 'eh iret a ']
inp ['autistic a', 'd jellying'] ans ['utistic ad', ' jellying '] pre ['n taiussma', ' taiussmai']
inp ['dults in a', ' food may '] ans ['ults in a ', 'food may b'] pre ['earoar d  ', 'aroar d  a']
inp [' local com', 'be preserv'] ans ['local comm', 'e preserve'] pre ['tea saerle', 'ea saerle ']
inp ['munity wil', 'ed by cook'] ans ['unity will', 'd by cooki'] pre ['urn ianei ', 'rn ianei  ']
inp ['l help out', 'ing in a m'] ans [' help out ', 'ng in a ma'] pre ['enaga aain', 'naga aaina']
inp [' children ', 'aterial th'] ans ['children w', 'terial tha'] pre ['anoeidl ln', 'noeidl ln ']
inp ['with autis', 'at solidif'] ans ['ith autism', 't solidifi'] pre ['itt ha uam', 'tt ha uam ']
inp ['m much in ', 'ies to for'] ans [' much in t', 'es to form'] pre [' casa eahh', 'casa eahh ']
inp ['the 

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---